# 에이전트(Agent)

에이전트는 언어 모델과 도구를 결합하여 작업에 대해 추론하고, 사용할 도구를 결정하며, 솔루션을 향해 반복적으로 작업할 수 있는 시스템을 만듭니다.

`create_agent`는 프로덕션 수준의 에이전트 구현을 제공합니다.

## 환경 설정

In [ ]:
from dotenv import load_dotenv
from langchain_teddynote import logging

# 환경 변수 로드
load_dotenv(override=True)
# 추적을 위한 프로젝트 이름 설정
logging.langsmith("LangChain-Advanced-Tutorial")

### 모델 (Model)

에이전트의 추론 엔진인 LLM 은 간단하게 `provider:model` 형식의 문자열로 지정할 수 있습니다.

In [ ]:
from langchain.agents import create_agent

# 모델 식별자 문자열을 사용한 간단한 방법
agent = create_agent("openai:gpt-4.1-mini", tools=[])

하지만, 모델의 세부 설정을 위해서 다음과 같이 다양한 옵션을 사용할 수 있습니다.

In [ ]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# 모델 인스턴스를 직접 초기화하여 더 세밀한 제어
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.1,  # 응답의 무작위성 제어
    max_tokens=1000,  # 최대 생성 토큰 수
    timeout=30,  # 요청 타임아웃(초)
)

agent = create_agent(model, tools=[])

#### 동적 모델

동적 모델은 런타임에 현재 상태와 컨텍스트를 기반으로 선택됩니다. 이를 통해 정교한 라우팅 로직과 비용 최적화가 가능합니다.

![](assets/wrap_model_call.png)

`ModelRequest`는 agent의 모델 호출 정보를 담는 dataclass로, middleware에서 요청을 검사하고 수정할 때 사용됩니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

# 기본 모델과 고급 모델 정의
basic_model = ChatOpenAI(model="gpt-4.1-mini")
advanced_model = ChatOpenAI(model="gpt-4.1")


@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """대화 복잡도에 따라 모델 선택"""
    message_count = len(request.state["messages"])

    # 긴 대화에는 고급 모델 사용
    if message_count > 10:
        model = advanced_model
    else:
        model = basic_model

    request.model = model
    return handler(request)


agent = create_agent(
    model=basic_model, tools=[], middleware=[dynamic_model_selection]  # 기본 모델
)

In [ ]:
from langchain_teddynote.messages import stream_graph
from langchain_core.messages import HumanMessage

stream_graph(
    agent,
    inputs={
        "messages": [HumanMessage(content="머신러닝의 동작 원리에 대해서 설명해줘")]
    },
)

수정에 활용할 **주요 속성**은 다음과 같습니다.

* `model`: 사용할 `BaseChatModel` 인스턴스
* `system_prompt`: 시스템 프롬프트 (optional)
* `messages`: 대화 메시지 리스트 (시스템 프롬프트 제외)
* `tool_choice`: tool 선택 설정
* `tools`: 사용 가능한 tool 리스트
* `response_format`: 응답 형식 지정
* `state`: 현재 agent 상태 (`AgentState`)
* `runtime`: agent runtime 정보
* `model_settings`: 추가 모델 설정 (dict)

In [ ]:
@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """대화 복잡도에 따라 모델 선택"""
    message_count = len(request.state["messages"][-1].content)
    print(f"글자수: {message_count}")

    # 긴 대화에는 고급 모델 사용
    if message_count > 10:
        # 여러 속성 동시 변경
        new_request = request.override(
            model=advanced_model,
            system_prompt="emoji 를 사용해서 답변해줘",
            tool_choice="auto",
        )
        return handler(new_request)
    else:
        new_request = request.override(
            system_prompt="한 문장으로 간결하게 답변해줘. emoji 는 사용하지 말아줘.",
            tool_choice="auto",
            model=basic_model,
        )
        return handler(new_request)


agent = create_agent(
    model=basic_model, tools=[], middleware=[dynamic_model_selection]  # 기본 모델
)

글자수 10자 미만일 때의 응답

In [ ]:
stream_graph(agent, inputs={"messages": [HumanMessage(content="머신러닝 동작원리")]})

글자수 10자 이상일 때의 응답

In [ ]:
stream_graph(
    agent,
    inputs={
        "messages": [
            HumanMessage(content="머신러닝의 동작 원리에 대해서 설명해 주세요.")
        ]
    },
)

## 프롬프트

### 시스템 프롬프트

`system_prompt` 매개변수를 사용하여 에이전트의 기본 동작을 정의할 수 있습니다.

In [ ]:
agent = create_agent(
    "openai:gpt-4.1-mini",
    system_prompt="You are a helpful assistant. Be concise and accurate.",
)

출력

In [ ]:
stream_graph(
    agent,
    inputs={"messages": [HumanMessage(content="대한민국의 수도는 어디야?")]},
)

### 동적 시스템 프롬프트(Dynamic Prompting)

런타임 컨텍스트나 에이전트 상태를 기반으로 시스템 프롬프트를 수정해야 하는 고급 사용 사례의 경우 `dynamic_prompt` 미들웨어를 사용할 수 있습니다.

In [ ]:
from typing import TypedDict
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest


class Context(TypedDict):
    prompt_type: str
    length: int


@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """사용자 역할에 따라 시스템 프롬프트 생성"""
    # 답변 형식 설정
    answer_type = (
        request.runtime.context.get("prompt_type", "default")
        if request.runtime.context
        else "default"
    )
    # 답변 길이 설정
    answer_length = (
        request.runtime.context.get("length", 20) if request.runtime.context else 20
    )
    base_prompt = "You are a helpful assistant. Answer in Korean.\n"

    # 답변 형식에 따라 시스템 프롬프트 생성(동적 프롬프팅)
    if answer_type == "default":
        return f"{base_prompt} [답변 형식] 간결하게 답변해줘. 답변 길이는 {answer_length}자 이하로 해줘."
    elif answer_type == "sns":
        return f"{base_prompt} [답변 형식] SNS 형식으로 답변해줘. 답변 길이는 {answer_length}자 이하로 해줘."
    elif answer_type == "article":
        return f"{base_prompt} [답변 형식] 뉴스 기사 형식으로 답변해줘. 답변 길이는 {answer_length}자 이하로 해줘."
    else:
        return f"{base_prompt} [답변 형식] 간결하게 답변해줘. 답변 길이는 {answer_length}자 이하로 해줘."


# 컨텍스트 스키마와 user_role_prompt 미들웨어를 사용하여 에이전트 생성
agent = create_agent(
    model="openai:gpt-4.1-mini",
    middleware=[user_role_prompt],
    context_schema=Context,
)

In [ ]:
# 컨텍스트에 따라 시스템 프롬프트가 동적으로 설정됩니다
stream_graph(
    agent,
    inputs={
        "messages": [HumanMessage(content="머신러닝의 동작 원리에 대해서 설명해줘")]
    },
    context=Context(prompt_type="article", length=1000),
)

In [ ]:
stream_graph(
    agent,
    inputs={
        "messages": [HumanMessage(content="머신러닝의 동작 원리에 대해서 설명해줘")]
    },
    context=Context(prompt_type="sns", length=50),
)

## 구조화된 답변 출력(Response Format)

특정 형식으로 에이전트의 출력을 반환하고 싶을 때가 있습니다. LangChain은 `response_format` 매개변수를 통해 구조화된 출력 전략을 제공합니다.

## Response Format 설정

`response_format` 파라미터는 구조화된 응답을 위한 선택적 설정입니다.

**지원 타입**

다음 세 가지 타입 중 하나를 사용할 수 있습니다

* `ToolStrategy`: Tool 기반 구조화 전략
* `ProviderStrategy`: Provider 기반 구조화 전략
* Pydantic model class: Pydantic 모델 클래스


**참고사항**

* 모델의 구조화된 출력 지원 여부에 따라 적절한 strategy가 선택됩니다
* 구조화된 응답은 대화 컨텍스트 내에서 자동으로 처리됩니다

### pydatic model 기반 처리 예시

In [ ]:
from pydantic import BaseModel, Field


class ContactInfo(BaseModel):
    """Response schema for the agent."""

    name: str = Field(description="The name of the person")
    email: str = Field(description="The email of the person")
    phone: str = Field(description="The phone number of the person")

In [ ]:
agent = create_agent(model="openai:gpt-4.1-mini", tools=[], response_format=ContactInfo)

result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Extract contact info from: 테디는 AI 엔지니어 입니다. 그의 이메일은 teddy@example.com 이고, 전화번호는 010-1234-5678 입니다.",
            }
        ]
    }
)

정형화된 출력

In [ ]:
result["structured_response"]

### ToolStrategy 예시

`ToolStrategy`는 도구 호출을 사용하여 구조화된 출력을 생성합니다. 도구 호출(Tool Calling)을 지원하는 모든 모델에서 작동합니다.

In [ ]:
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


# 응답 스키마 정의
class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str


agent = create_agent(
    model="openai:gpt-4.1-mini", tools=[], response_format=ToolStrategy(ContactInfo)
)

result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Extract contact info from: 테디는 AI 엔지니어 입니다. 그의 이메일은 teddy@example.com 이고, 전화번호는 010-1234-5678 입니다.",
            }
        ]
    }
)

result["structured_response"]

### ProviderStrategy 사용 예시

`ProviderStrategy`는 모델 제공자의 네이티브 구조화된 출력 생성을 사용합니다. 

더 안정적이지만 네이티브 구조화된 출력을 지원하는 제공자(예: OpenAI)에서만 작동합니다.

In [ ]:
from langchain.agents.structured_output import ProviderStrategy

agent = create_agent(
    model="openai:gpt-4.1", response_format=ProviderStrategy(ContactInfo)
)

In [ ]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Extract contact info from: 테디는 AI 엔지니어 입니다. 그의 이메일은 teddy@example.com 이고, 전화번호는 010-1234-5678 입니다.",
            }
        ]
    }
)

In [ ]:
result["structured_response"]

### 미들웨어를 통한 중간 상태 제어

`before_model` 및 `after_model` 미들웨어는 모델 호출 전후에 중간 상태를 제어할 수 있는 훅입니다.

In [ ]:
from langchain.agents.middleware import (
    before_model,
    after_model,
)
from langchain.agents.middleware import (
    AgentState,
    ModelRequest,
    ModelResponse,
    dynamic_prompt,
)
from langchain.chat_models import init_chat_model
from langchain.messages import AIMessage, AnyMessage
from langchain_teddynote.messages import invoke_graph
from langchain_core.prompts import PromptTemplate
from langgraph.runtime import Runtime
from typing import Any, Callable


# 노드 스타일: 모델 호출 전 로깅
@before_model
def log_before_model(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    print(
        f"\033[95m\n\n모델 호출 전 메시지 {len(state['messages'])}개가 있습니다\033[0m"
    )
    last_message = state["messages"][-1].content
    llm = init_chat_model("openai:gpt-4.1-mini")

    query_rewrite = (
        PromptTemplate.from_template(
            "Rewrite the following query to be more understandable. Do not change the original meaning. Make it one sentence: {query}"
        )
        | llm
    )
    rewritten_query = query_rewrite.invoke({"query": last_message})

    return {"messages": [rewritten_query.content]}


@after_model
def log_after_model(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:

    print(
        f"\033[95m\n\n모델 호출 후 메시지 {len(state['messages'])}개가 있습니다\033[0m"
    )
    for i, message in enumerate(state["messages"]):
        print(f"[{i}] {message.content}")
    return None

In [ ]:
agent = create_agent(
    "openai:gpt-4.1-mini",
    middleware=[
        log_before_model,
        log_after_model,
    ],
)

In [ ]:
stream_graph(
    agent,
    inputs={"messages": [HumanMessage(content="대한민국 수도")]},
)

### Class 기반 미들웨어 사용

데코레이터 대신 클래스 기반 미들웨어를 사용할 수 있습니다.

오버라이드 해야 하는 메서드는 `before_model` 및 `after_model` 메서드입니다.

In [ ]:
from typing import Any
from langchain.agents import AgentState
from langchain.agents.middleware import AgentMiddleware


# 커스텀 상태 스키마 정의
class CustomState(AgentState):
    user_preferences: dict


class CustomMiddleware(AgentMiddleware):
    state_schema = CustomState
    tools = []

    def before_model(self, state: CustomState, runtime) -> dict[str, Any] | None:
        # 모델 호출 전 커스텀 로직
        pass


agent = create_agent("openai:gpt-4.1-mini", tools=[], middleware=[CustomMiddleware()])

# 에이전트는 이제 메시지 외에 추가 상태를 추적할 수 있습니다
result = agent.invoke(
    {
        "messages": [{"role": "user", "content": "I prefer technical explanations"}],
        "user_preferences": {"style": "technical", "verbosity": "detailed"},
    }
)

### 모델 오류시 재시도 로직

In [ ]:
@wrap_model_call
def retry_model(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse],
) -> ModelResponse:
    for attempt in range(3):
        try:
            return handler(request)
        except Exception as e:
            if attempt == 2:
                raise
            print(f"오류 발생으로 {attempt + 1}/3 번째 재시도합니다: {e}")

In [ ]:
agent = create_agent(
    "openai:gpt-4.1-minis",  # 일부러 모델 호출 실패하도록 설정(모델명 오류)
    middleware=[retry_model],
)

In [ ]:
stream_graph(
    agent,
    inputs={"messages": [HumanMessage(content="대한민국의 수도는?")]},
)